In [30]:
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role


In [31]:
role = sagemaker.get_execution_role()  # auto-fills execution role
bucket = 'sagemaker-us-east-1-xxxx'  # change this
model_key = 'model.tar.gz'   # change if you uploaded to a subfolder
model_uri = f's3://{bucket}/{model_key}'


In [32]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
import sagemaker

role = get_execution_role()
sagemaker_session = sagemaker.Session()

model = PyTorchModel(
    model_data='s3://sagemaker-us-east-1-xxxx/model.tar.gz',
    role=role,
    entry_point='inference.py',
    source_dir='model/code',
    framework_version='1.12.1',
    py_version='py38',
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='xxxx.xxxx'     # name endpoint
)



------!

In [33]:
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from sagemaker.predictor import Predictor
import io

# 1. Load your real image (replace with your image path)
img_path = 'aeroplane.jpg'  # or .png etc.
image = Image.open(img_path).convert("RGB")  # ensure it's RGB

# 2. Define preprocessing transform (resize + normalize like CIFAR-10)
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # resize to CIFAR-10 size
    transforms.ToTensor(),  # convert to torch tensor (C, H, W)
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # normalize to [-1, 1]
])

# 3. Apply transform and convert to numpy array
image_tensor = transform(image)
img_array = image_tensor.unsqueeze(0).numpy()  # add batch dim, shape = (1, 3, 32, 32)

# 4. Convert to bytes for x-npy format
npy_bytes = io.BytesIO()
np.save(npy_bytes, img_array)
npy_bytes.seek(0)

# 5. Create Predictor and send request
predictor = Predictor(endpoint_name='xxxx.xxxx')  # your deployed endpoint name

response = predictor.predict(npy_bytes.read(), {
    "ContentType": "application/x-npy",
    "Accept": "application/json"
})

# 6. Output the prediction
print("Predicted class index:", response)


Predicted class index: b'{"predicted_class": 0, "confidence": 1.0}'
